# <div align="center">Detail understanding of SMPL paper</div>
---------------------------------------------------------------------

you can Find me on Github:
> ###### [ GitHub](https://github.com/lev1khachatryan)


Skinned Multi-Person Linear model (SMPL) is a ***skinned vertex-based*** model that accurately represents a wide variety of body shapes in natural human poses. The parameters of the model are learned from data including the ***rest pose template***, ***blend weights***, ***pose-dependent blend shapes***, ***identity-dependent blend shapes***, and a ***regressor from vertices to joint locations***. Unlike previous models, the pose-dependent blend shapes are a linear function of the elements of the pose rotation matrices. This simple formulation enables training the entire model from a relatively large number of aligned 3D meshes of different people in different poses. We quantitatively evaluate variants of SMPL using linear or dual-quaternion
blend skinning and show that both are more accurate than a BlendSCAPE model trained on the same data. We also extend SMPL to
realistically model dynamic soft-tissue deformations. Because it is based on blend skinning, SMPL is compatible with existing rendering engines and we make it available for research purposes.

The surprise is that a ***vertex-based***, skinned, model such as SMPL is actually more accurate than a ***deformation-based*** model like BlendSCAPE trained on the same data. The test meshes are available for research purposes so others can quantitatively compare to SMPL.

## <div align="center">Model Formulation</div>
---------------------------------------------------------------------

The model is defined by a mean template shape represented by a vector of N concatenated vertices $\overline{T} \in R^{3N}$ in the zero pose, $\theta^{*}$ a set of blend weights, $\Omega \in R^{NxK}$, ($\Omega$ is math W) (Fig. 1(a)), a blend shape function, $B_{S}(\beta): R^{|\beta|} \to R^{3N}$, that takes as input a vector of shape parameters, $\beta$, (Fig. 1(b)) and outputs a blend shape sculpting the subject identity; a function to predict K joint locations (white dots in Fig. 1), $J(\beta): R^{|\beta|} \to R^{3K}$, as a function of shape parameters, $\beta$; and a pose-dependent blend shape function, $B_{p}(\theta): R^{|\theta|} \to R^{3N}$, that takes as input a vector of pose parameters, $\theta$, and accounts for the effects of posedependent deformations (Fig. 1(c)). The corrective blend shapes of these functions are added together in the rest pose as illustrated in
(Fig. 1(c)). Finally, a standard blend skinning function $W$ (linear or dual-quaternion) is applied to rotate the vertices around the estimated joint centers with smoothing defined by the blend weights. 

The result is a model, $M(\beta, \theta; \Phi): R^{|\theta|x|\beta|} \to R^{3N}$, that maps shape and pose parameters to vertices (Fig. 1(d)). Here $\Phi$ represents the learned model parameters.

<img src='_assets/main.png'>

***Figure 1: SMPL model.*** (a) Template mesh with blend weights indicated by color and joints shown in white. (b) With identity-driven blendshape contribution only; vertex and joint locations are linear in shape vector $\beta$. (c) With the addition of of pose blend shapes in preparation for the split pose; note the expansion of the hips. (d) Deformed vertices reposed by dual quaternion skinning for the split pose.

### Blend skinning

The pose of the body is defined by a standard skeletal rig, where $w_k \in R^3$  denotes the axis-angle representation of the relative rotation of part k with respect to its parent in the kinematic tree. Our rig has K = 23 joints, hence a pose $\theta = [w_0^T, ..., w_k^T]$ is defined by $|\theta|$ = 3 x 23 + 3 = 72 parameters; i.e. 3 for each part plus 3 for the root orientation. Let $\overline{w} = w / ||w||$ denote the unit norm axis of rotation. Then the axis angle for every joint j is transformed to a rotation matrix using the ***Rodrigues formula*** $exp(w_j) = I + \hat{\overline{w_j}} sin(||w_j||) + \hat{\overline{w_j}}^2 cos(||w_j||)$, where $\hat{\overline{w}}$ is the skew symmetric matrix of the 3-vector $\overline{w}$ and $I$ is the $3x3$ identity matrix.

Using this, the standard linear blend skinning function $W(\overline{T}, J, \theta, \Omega): R^{3Nx3Kx|\theta|x|\Omega|} \to R^{3N}$ takes vertices in the rest pose, $\overline{T}$, joint locations, J, a pose, $\theta$, and the blend weights, $\Omega$, and returns the posed vertices. Each vertex $\bar{t_i}$ in $\overline{T}$ is transformed into $\bar{t_i}'$ (both column vectors in homogeneous coordinates) as 

<img src='_assets/6.png'>

where $w_{k,i}$ is an element of the blend weight matrix $\Omega$, representing how much the rotation of part k effects the vertex i, $exp(w_j)$ is the local $3x3$ rotation matrix corresponding to joint j, $G_k(\theta, J)$ is
the world transformation of joint k, and $G_k'(\theta, J)$ is the same transformation after removing the transformation due to the rest pose, $\theta^*$. Each 3-element vector in J corresponding to a single joint center, j, is denoted $j_j$ . Finally, $A(k)$ denotes the ordered set of joint ancestors of joint k. Note, for compatibility with existing rendering engines, we assume $\Omega$ is sparse and allow at most four parts to influence a vertex.

To maintain compatibility, we keep the basic skinning function and instead modify the template in an additive way and learn a function to predict joint locations. Our model, $M(\beta, \theta)$ is then 

<img src='_assets/7.png'>

where $B_S(\beta)$ and $B_P(\theta)$ are vectors of vertices representing offsets from the template. We refer to these as ***shape and pose blend shapes*** respectively. Given this definition, a vertex $\bar{t_i}$ is transformed according to 

<img src='_assets/8.png'>